# Training ```FFANN```

Let's augment the ```FFANN``` class in order to be able to train it.

This is still a work in progress, but at least it can "fit" the *XOR*-gate and a linear function.

In [1]:
import FeedForwardANN as FFANN
import numpy as np

In [2]:
lin=FFANN.linearActivation()
sig=FFANN.sigmoidActivation()



In [3]:
# data_in=[[_] for _ in np.linspace(0,1,250)]
# data_out=[[2*_] for _ in np.linspace(0,1,250)]
# brain=FFANN.FFANN(1,1,[3,2,1],[lin,lin,lin,lin,lin])
# brain.init_params(-1,1)


data_in=[[1,1],[0,0],[1,0],[0,1]]
data_out=[[0],[0],[1],[1]]
brain=FFANN.FFANN(2,1,[3],[sig,lin])
brain.init_params(-1,1)

In [4]:
def Q_i(signal,target):
    return (signal-target)**2.

def dQds_i(signal,target):
    return 2.*(signal-target)

In [5]:
Q=FFANN.loss(Q_i, dQds_i, brain)
# strategy=FFANN.VanillaSGD(brain,Q,alpha=1e-1)
strategy=FFANN.RMSpropSGD(brain,Q,gamma=0.995,epsilon=1e-5,alpha=1e-2)

In [6]:
brain.SGD(strategy, data_in, data_out, abs_tol=1e-2, rel_tol=1e-3, step_break=150,max_step=15000)

In [7]:
meanQ=0
maxQ=0
for i,_ in enumerate(data_in):
    Qi=Q(brain(_),data_out[i])
    print('point:',_,'\n'
       'target:',data_out[i],
         'FFANN gives:',brain(_),
         'loss:',Qi)
    print('')
    if Qi > maxQ:
        maxQ=Qi
    meanQ+=Q(brain(_),data_out[i])
    
print('mean loss:', meanQ/float(len(data_out)))
print('max loss:', maxQ)


point: [1, 1] 
target: [0] FFANN gives: [0.00014472567547496595] loss: 2.094552114168516e-08

point: [0, 0] 
target: [0] FFANN gives: [-0.0010851722689637633] loss: 1.1775988533279622e-06

point: [1, 0] 
target: [1] FFANN gives: [0.9999851685794274] loss: 2.1997103620044367e-10

point: [0, 1] 
target: [1] FFANN gives: [1.0000688153222308] loss: 4.735548573728486e-09

mean loss: 3.008749735198941e-07
max loss: 1.1775988533279622e-06
